## Imports

In [1]:
import pandas as pd
import numpy as np
import random
import csv
import os
import json
from tqdm import tqdm
import math
import altair as alt
import glob
import sys

module_path = os.path.abspath(os.path.join('..', 'scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)
#end if

import utils
from utils import read_json

## Declares

In [2]:
TAG = "mammalian_REM2"
DATA_DIRECTORY = os.path.join("..", "results", TAG)

JSONS_BS = glob.glob(os.path.join(DATA_DIRECTORY, "*.BUSTEDS.json"))
JSONS_BSMH = glob.glob(os.path.join(DATA_DIRECTORY, "*.BUSTEDS+MH.json"))
JSONS_BASE = glob.glob(os.path.join(DATA_DIRECTORY, "*.BUSTED.json"))
JSONS_BMH = glob.glob(os.path.join(DATA_DIRECTORY, "*.BUSTED+MH.json"))

# Create tables folder
OUTPUT_CSV = TAG + "_BUSTED_ModelTest.csv"

ER_Threshold = 10
ER_Threshold_loose = 1

pval_Threshold = 0.05

Tests = 4

In [3]:
#define function to calculate cv
#cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
cv = lambda x: np.std(x) / np.mean(x)
pctchg = lambda a, b: (a / b) * 100

"""
def read_json(filename):
    if os.stat(filename).st_size == 0: 
        print("# -- Error -- file is empty:", filename)
        return []
    #end if
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    return json_data
#end method
"""

def convergence_issue(json_data):
    # For example if logL for a more complex model is LOWER than it is for a less complex model.
    convergence_issue = False 
    return ""
    """
    Constrained model
    MG94xREV with separate rates for branch sets
    Nucleotide GTR
    Unconstrained model
    """
    
    _models = ["Nucleotide GTR", 
                    "MG94xREV with separate rates for branch sets", 
                    "Constrained model", 
                    "Unconstrained model"]
    model_LL = {}
    
    for _model in _models:
        print(_model)
        _LL = json_data["fits"][_model]["Log Likelihood"]

        
    """
    return
    #Nucleotide_GTR = BS_data["fits"]["Nucleotide GTR"].get("Log Likelihood", 10)
    #MG94_REV = BS_data["fits"]["MG94xREV with separate rates for branch sets"].get("Log Likelihood",10)
    try:
        Constrained_model = BS_data["fits"]["Constrained model"].get("Log Likelihood", 10)
    except:
        Constrained_model = 10
    #end try
    Unconstrained_model = BS_data["fits"]["Unconstrained model"].get("Log Likelihood", 10)
    if Nucleotide_GTR < MG94_REV and MG94_REV < Constrained_model and Constrained_model < Unconstrained_model:
        pass
    else:
        issue = True
    #end if
    if issue == False:
        return ""
    else:
        return "ISSUE DETECTED"
    #end if
    """
#end method




def convergence_issue_old(BS_data):
    # For example if logL for a more complex model is LOWER than it is for a less complex model.
    convergence_issue = False 
    
    #Nucleotide_GTR = BS_data["fits"]["Nucleotide GTR"].get("Log Likelihood", 10)
    #MG94_REV = BS_data["fits"]["MG94xREV with separate rates for branch sets"].get("Log Likelihood",10)
    
    try:
        Constrained_model = BS_data["fits"]["Constrained model"].get("Log Likelihood", 10)
    except:
        Constrained_model = 10
    #end try
    
    Unconstrained_model = BS_data["fits"]["Unconstrained model"].get("Log Likelihood", 10)
    
    if Nucleotide_GTR < MG94_REV and MG94_REV < Constrained_model and Constrained_model < Unconstrained_model:
        pass
    else:
        issue = True
    #end if
    
    if issue == False:
        return ""
    else:
        return "ISSUE DETECTED"
    #end if
#end method

In [4]:
def process(FILES, fileending, method, pval_Threshold, Tests):
    df_dict = {}
    Bonferroni_pval = pval_Threshold / Tests
    for item in FILES:
        print("\t Processing:", item)
        basename = ""
        if fileending in os.path.basename(item):
             basename = os.path.basename(item).replace(fileending, "").replace(".best-gard", "")
             #basename = os.path.basename(item)
             #print("\t Basename:", basename)
        #end if
        for fext in [".phy", ".fasta", ".nex", "-align-dna.fas", "-Aligned-DNA.fas"]:
            basename = basename.replace(fext, "")
        #end for
        json_data = read_json(item) # Read json
        if json_data == []:
            continue # Empty file
        #end if
        df_dict[basename] = {"Method": method}
        df_dict[basename].update({"Sequences": json_data["input"]["number of sequences"]})
        df_dict[basename].update({"Codons": json_data["input"]["number of sites"]})
        df_dict[basename].update({"LRT p-value": json_data["test results"]["p-value"]})
        df_dict[basename].update({"Bonferroni p-value": Bonferroni_pval})
        Bonferroni_sig = False
        if json_data["test results"]["p-value"] <= Bonferroni_pval:
            Bonferroni_sig = True
        #end if
        # Bonferroni Test
        df_dict[basename].update({"Bonferroni significant": str(Bonferroni_sig)})
        # cAIC
        df_dict[basename].update({"cAIC": json_data["fits"]["Unconstrained model"]["AIC-c"]})
        #Omegas and proportions
        data = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]
        w1 = data["0"]["omega"]
        p1 = data["0"]["proportion"]
        w2 = data["1"]["omega"]
        p2 = data["1"]["proportion"]
        w3 = data["2"]["omega"]
        p3 = data["2"]["proportion"]
        df_dict[basename].update({"w1": w1, "p1": p1})
        df_dict[basename].update({"w2": w2, "p2": p2})
        df_dict[basename].update({"w3": w3, "p3": p3})
        # SRV rates and proportions
        if method == "BUSTEDS+MH" or method == "BUSTEDS":
            data = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
            s1 = data["0"]["rate"]
            s_p1 = data["0"]["proportion"]
            s2 = data["1"]["rate"]
            s_p2 = data["1"]["proportion"]
            s3 = data["2"]["rate"]
            s_p3 = data["2"]["proportion"]
            df_dict[basename].update({"SRV1": s1, "SRV_p1": s_p1})
            df_dict[basename].update({"SRV2": s2, "SRV_p2": s_p2})
            df_dict[basename].update({"SRV3": s3, "SRV_p3": s_p3})
        #end if
        # DH rate, TH rate, TH_SI rate
        if method == "BUSTEDS+MH" or method == "BUSTED+MH":
            df_dict[basename].update({"DH_Rate": float(json_data["fits"]["Unconstrained model"]["Rate Distributions"]["rate at which 2 nucleotides are changed instantly within a single codon"])})
            df_dict[basename].update({"TH_Rate": float(json_data["fits"]["Unconstrained model"]["Rate Distributions"]["rate at which 3 nucleotides are changed instantly within a single codon"])})
            #df_dict[basename].update({"TH_Rate_SI": float(json_data["fits"]["Unconstrained model"]["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"])})
        #end if
        # Convergence issues check
        df_dict[basename].update({"Convergence_Issue": convergence_issue(json_data)})
    # end for
    df = pd.DataFrame.from_dict(df_dict, orient="index")
    df = df.reset_index()
    df.index += 1
    df.rename(columns={'index': 'Gene'}, inplace = True)
    return df
#end method


## Process

In [5]:
print("# Processing BUSTED[S] files")
df_BUSTEDS = process(JSONS_BS , ".BUSTEDS.json", "BUSTEDS", pval_Threshold, Tests)

print("\n# Processing BUSTED[S]-MH files")
df_BUSTEDS_MH = process(JSONS_BSMH, ".BUSTEDS+MH.json", "BUSTEDS+MH", pval_Threshold, Tests)

print("\n# Processing BUSTED files")
df_BUSTED = process(JSONS_BASE , ".BUSTED.json", "BUSTED", pval_Threshold, Tests)
print("\n# Processing BUSTED-MH files")
df_BUSTED_MH = process(JSONS_BMH, ".BUSTED+MH.json", "BUSTED+MH", pval_Threshold, Tests)

# Processing BUSTED[S] files
	 Processing: ../results/mammalian_REM2/mammalian_REM2_codons.SA.FilterOutliers.fasta.BUSTEDS.json

# Processing BUSTED[S]-MH files
	 Processing: ../results/mammalian_REM2/mammalian_REM2_codons.SA.FilterOutliers.fasta.BUSTEDS+MH.json

# Processing BUSTED files
	 Processing: ../results/mammalian_REM2/mammalian_REM2_codons.SA.FilterOutliers.fasta.BUSTED.json

# Processing BUSTED-MH files
	 Processing: ../results/mammalian_REM2/mammalian_REM2_codons.SA.FilterOutliers.fasta.BUSTED+MH.json


## Concatenate tables


In [6]:
result = pd.concat([df_BUSTEDS, df_BUSTEDS_MH, df_BUSTED, df_BUSTED_MH])
result = result.fillna("")
result = result.sort_values(by=["Gene", "Method"], ascending=True)
result = result.reset_index(drop=True)
result.index += 1
result

,Gene,Method,Sequences,Codons,LRT p-value,Bonferroni p-value,Bonferroni significant,cAIC,w1,p1,...,p3,SRV1,SRV_p1,SRV2,SRV_p2,SRV3,SRV_p3,Convergence_Issue,DH_Rate,TH_Rate
1,mammalian_REM2_codons.SA.FilterOutliers,BUSTED,175,627,9.032219e-13,0.0125,True,47174.746248,0.000000,0.695806,...,0.002933,,,,,,,,,
2,mammalian_REM2_codons.SA.FilterOutliers,BUSTED+MH,175,627,1.907041e-03,0.0125,True,47168.799678,0.064673,0.928514,...,0.001389,,,,,,,,0.04478,0.06523
3,mammalian_REM2_codons.SA.FilterOutliers,BUSTEDS,175,627,2.024492e-13,0.0125,True,46351.638677,0.122135,0.995947,...,0.001154,0.498937,0.551292,1.335249,0.379864,3.162586,0.068845,,,
4,mammalian_REM2_codons.SA.FilterOutliers,BUSTEDS+MH,175,627,5.000000e-01,0.0125,False,46352.004222,0.100309,0.612875,...,0.025081,0.493873,0.541742,1.322228,0.385386,3.05851,0.072872,,0.055794,0.259925


## Calculate AIC weighted p-values

In [7]:
df_main_holder = []
for gene in set(result["Gene"].tolist()):
    print(gene)
    df_holder = result[result["Gene"] == gene]
    min_cAIC = min(df_holder["cAIC"].tolist())
    print("\tBest cAIC is:", min_cAIC)
    averaged = {}
    for index, row in df_holder.iterrows():
        method = row["Method"]
        cAIC   = row["cAIC"]
        pvalue = row["LRT p-value"]
        weight = math.exp (-0.5 * (cAIC - min_cAIC))
        if cAIC == min_cAIC:
            print("\tBest method:", method)
        #end if
        averaged[method] = {
                            "weight": weight,
                            "original_pval": pvalue
        }
    #end for
    weight_sum = 0
    for method in averaged.keys():
        weight_sum += averaged[method]["weight"]
    #end for
    for method in averaged.keys():
        averaged[method]["normalized_weighted"] = averaged[method]["weight"] / weight_sum
    #end for
    #calculate p_ma
    p_ma = 0
    for method in averaged.keys():
        p_ma += averaged[method]["original_pval"] * averaged[method]["normalized_weighted"]
    #end for
    print("p_ma is:", p_ma)
    df_holder["p_value_averaged"] = p_ma
    df_main_holder.append(df_holder)
#end for

result = pd.concat(df_main_holder)
result = result.fillna("")
result = result.sort_values(by=["Gene", "Method"], ascending=True)
result = result.reset_index(drop=True)
result.index += 1


mammalian_REM2_codons.SA.FilterOutliers
	Best cAIC is: 46351.63867737226
	Best method: BUSTEDS
p_ma is: 0.22721687386546213


In [8]:
result

,Gene,Method,Sequences,Codons,LRT p-value,Bonferroni p-value,Bonferroni significant,cAIC,w1,p1,...,SRV1,SRV_p1,SRV2,SRV_p2,SRV3,SRV_p3,Convergence_Issue,DH_Rate,TH_Rate,p_value_averaged
1,mammalian_REM2_codons.SA.FilterOutliers,BUSTED,175,627,9.032219e-13,0.0125,True,47174.746248,0.000000,0.695806,...,,,,,,,,,,0.227217
2,mammalian_REM2_codons.SA.FilterOutliers,BUSTED+MH,175,627,1.907041e-03,0.0125,True,47168.799678,0.064673,0.928514,...,,,,,,,,0.04478,0.06523,0.227217
3,mammalian_REM2_codons.SA.FilterOutliers,BUSTEDS,175,627,2.024492e-13,0.0125,True,46351.638677,0.122135,0.995947,...,0.498937,0.551292,1.335249,0.379864,3.162586,0.068845,,,,0.227217
4,mammalian_REM2_codons.SA.FilterOutliers,BUSTEDS+MH,175,627,5.000000e-01,0.0125,False,46352.004222,0.100309,0.612875,...,0.493873,0.541742,1.322228,0.385386,3.05851,0.072872,,0.055794,0.259925,0.227217


In [9]:
#for method in averaged.keys():
#    averaged[method]["normalized_weighted"] = 
averaged

{'BUSTED': {'weight': 1.838493118342009e-179,
  'original_pval': 9.032219416837961e-13,
  'normalized_weighted': 1.0030198003967425e-179},
 'BUSTED+MH': {'weight': 3.5953684039232464e-178,
  'original_pval': 0.001907041306329127,
  'normalized_weighted': 1.9615116656558484e-178},
 'BUSTEDS': {'weight': 1.0,
  'original_pval': 2.024491685403973e-13,
  'normalized_weighted': 0.5455662522692967},
 'BUSTEDS+MH': {'weight': 0.832957951193782,
  'original_pval': 0.5,
  'normalized_weighted': 0.4544337477307034}}

In [10]:
df_holder = []

for g in set(result["Gene"].to_list()):
    #print(g)
    df_subset = result[result["Gene"] == g]
    df_subset["Annotation"] = ""
    min_cAIC = min(df_subset["cAIC"].to_list())
    if len(df_subset["cAIC"].to_list()) > 1:
        second_smallest = sorted(df_subset["cAIC"].to_list())[1]
    else:
        second_mallest = 0
    #end if
    for index, row in df_subset.iterrows():
        caic = row["cAIC"]
        if caic == min_cAIC:
            if second_smallest - caic > 5:
                df_subset["Annotation"][index] = "Strongly Preferred"
            else:
                df_subset["Annotation"][index] = "Preferred"
        #end if
    #end for
    df_holder.append(df_subset)
#end for
df_main = pd.concat(df_holder)
df_main = df_main.reset_index(drop=True)
df_main.index += 1
#df_main

/var/folders/3l/76gtv9qd0v3_8hn3cb7hfbs40000gn/T/ipykernel_63887/2835796267.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["Annotation"][index] = "Preferred"


## Save table

In [11]:
#OUTPUT_CSV = "Ebola.csv"
#OUTPUT_CSV = "Zika.csv"
#OUTPUT_CSV = "Monkeypox.csv"
#OUTPUT_CSV = TAG + "-internal.csv"

print("Saving results to:", OUTPUT_CSV)
#result = result.T

result.T.to_csv(os.path.join("..",
                           "tables",
                           OUTPUT_CSV))

Saving results to: mammalian_REM2_BUSTED_ModelTest.csv


In [12]:
result

,Gene,Method,Sequences,Codons,LRT p-value,Bonferroni p-value,Bonferroni significant,cAIC,w1,p1,...,SRV1,SRV_p1,SRV2,SRV_p2,SRV3,SRV_p3,Convergence_Issue,DH_Rate,TH_Rate,p_value_averaged
1,mammalian_REM2_codons.SA.FilterOutliers,BUSTED,175,627,9.032219e-13,0.0125,True,47174.746248,0.000000,0.695806,...,,,,,,,,,,0.227217
2,mammalian_REM2_codons.SA.FilterOutliers,BUSTED+MH,175,627,1.907041e-03,0.0125,True,47168.799678,0.064673,0.928514,...,,,,,,,,0.04478,0.06523,0.227217
3,mammalian_REM2_codons.SA.FilterOutliers,BUSTEDS,175,627,2.024492e-13,0.0125,True,46351.638677,0.122135,0.995947,...,0.498937,0.551292,1.335249,0.379864,3.162586,0.068845,,,,0.227217
4,mammalian_REM2_codons.SA.FilterOutliers,BUSTEDS+MH,175,627,5.000000e-01,0.0125,False,46352.004222,0.100309,0.612875,...,0.493873,0.541742,1.322228,0.385386,3.05851,0.072872,,0.055794,0.259925,0.227217


In [13]:
gene_list = []
for item in df_main["Gene"].to_list():
    gene_list.append(item.split("_")[0])
gene_set = set(gene_list)
#gene_set

In [14]:
df_main["Group"] = ""
for index, row in df_main.iterrows():
    gene = row["Gene"]
    df_main["Group"][index] = str(gene.split("_")[0])
df_main

/var/folders/3l/76gtv9qd0v3_8hn3cb7hfbs40000gn/T/ipykernel_63887/3009917599.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main["Group"][index] = str(gene.split("_")[0])


,Gene,Method,Sequences,Codons,LRT p-value,Bonferroni p-value,Bonferroni significant,cAIC,w1,p1,...,SRV2,SRV_p2,SRV3,SRV_p3,Convergence_Issue,DH_Rate,TH_Rate,p_value_averaged,Annotation,Group
1,mammalian_REM2_codons.SA.FilterOutliers,BUSTED,175,627,9.032219e-13,0.0125,True,47174.746248,0.000000,0.695806,...,,,,,,,,0.227217,,mammalian
2,mammalian_REM2_codons.SA.FilterOutliers,BUSTED+MH,175,627,1.907041e-03,0.0125,True,47168.799678,0.064673,0.928514,...,,,,,,0.04478,0.06523,0.227217,,mammalian
3,mammalian_REM2_codons.SA.FilterOutliers,BUSTEDS,175,627,2.024492e-13,0.0125,True,46351.638677,0.122135,0.995947,...,1.335249,0.379864,3.162586,0.068845,,,,0.227217,Preferred,mammalian
4,mammalian_REM2_codons.SA.FilterOutliers,BUSTEDS+MH,175,627,5.000000e-01,0.0125,False,46352.004222,0.100309,0.612875,...,1.322228,0.385386,3.05851,0.072872,,0.055794,0.259925,0.227217,,mammalian


## Visualize

In [15]:
result = result.rename(columns={"LRT p-value": "p_value"})
df_main = df_main.rename(columns={"LRT p-value": "p_value"})

In [16]:
source = df_main
pval_threshold = 0.05 / Tests

heatmap = alt.Chart(source).mark_rect(opacity=0.9).encode(
    y = alt.Y('Gene'),
    x = alt.X('Method', axis=alt.Axis(labelAngle=-60)),
    color = alt.condition((alt.datum.p_value <= pval_threshold), 
                                alt.ColorValue('darkblue'), 
                                alt.ColorValue('lightgray')) 
).properties(
    width=600,
    height=1200
)

text = heatmap.mark_text().encode(
    text='Annotation:O',
    color=alt.value('white'))

chart = alt.layer(heatmap, text).configure_axis(
    labelFontSize=12,
    titleFontSize=12,
    labelLimit = 1000
).configure_scale(bandPaddingInner=0.05).properties(
    title=TAG+"-InternalBranches Genome Scan with BUSTED, LRT pValue threshold: " + str(pval_threshold))

chart

alt.LayerChart(...)

In [17]:
source = df_main

pval_threshold = 0.05

heatmap = alt.Chart(source).mark_rect(opacity=0.75).encode(
    y = alt.Y('Gene'),
    x = alt.X('Method', axis=alt.Axis(labelAngle=-60)),
    color = alt.condition((alt.datum.p_value_averaged <= pval_threshold), 
                                alt.ColorValue('darkblue'), 
                                alt.ColorValue('gray')) 
).properties(
    width=800,
    height=1200
)

text = heatmap.mark_text().encode(
    text='Annotation:O',
    color=alt.value('white'))

chart = alt.layer(heatmap, text).configure_axis(
    labelFontSize=12,
    titleFontSize=12,
    labelLimit = 1000
).configure_scale(bandPaddingInner=0.05).properties(
    title=TAG+" on Internal Branches Genome Scan with BUSTED, LRT p-value threshold: " + str(pval_threshold) + " on the AIC-weighted pvalue ")

chart

alt.LayerChart(...)

In [18]:
source = df_main.copy()

pval_threshold = 0.05

heatmap = alt.Chart(source).mark_rect(opacity=0.75).encode(
    y = alt.Y('Gene'),
    x = alt.X('Method', axis=alt.Axis(labelAngle=-60)),
    color = alt.condition((alt.datum.p_value_averaged <= pval_threshold), 
                                alt.ColorValue('darkblue'), 
                                alt.ColorValue('orange')),
).properties(
    width=800,
    height=1200
)

text = heatmap.mark_text().encode(
    text='Annotation:O',
    color=alt.value('white'))

chart = alt.layer(heatmap, text).encode(
).configure_axis(
    labelFontSize=12,
    titleFontSize=12,
    labelLimit = 1000,
).configure_scale(bandPaddingInner=0.05).properties(
    title=TAG+" on Internal Branches Genome Scan with BUSTED, LRT p-value threshold: " + str(pval_threshold) + " on the AIC-weighted pvalue ")

chart

alt.LayerChart(...)